Currently, secrets are only persisted for the lifetime of the engine process and they are not part of the dprep file. If you started a new session (hence start an engine process) and load a package and try to run a dataflow within that package, you will need to call `use_secrets` to register the required secrets to use during execution, otherwise the execution will fail as the required secrets are not available.

In this notebook, we will:
1. Loading a previously saved package
2. Call `get_missing_secrets` to determine the missing secrets
3. Call `use_secrets` and pass in the missing secrets to register it with the engine for this session
4. Call `head` to see the a preview of the data

# Providing Secrets
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.

In [1]:
!pip install azureml

In [2]:
import azureml.dataprep as dprep
import os

Let's load the previously saved package.

In [3]:
package = dprep.Package.open(file_path='./data/secrets.dprep')
dataflow = package.dataflows[0]

Let's call `get_missing_secrets` to see what secrets are required missing in the engine.

In [4]:
dataflow.get_missing_secrets()

['https://dpreptestfiles.blob.core.windows.net/testfiles/read_csv_duplicate_headers.csv']

Let's now read the secrets from an environment variable and put it in our secret dictionary and call `use_secrets` with the secrets. This will register these secrets in the engine so you don't need to provide them again in this session.

_Note: It is a bad practice to have secrets in files that will be checked into source control._

In [5]:
sas = os.environ['SCENARIOS_SECRETS']
secrets = {
    'https://dpreptestfiles.blob.core.windows.net/testfiles/read_csv_duplicate_headers.csv': sas
}
dataflow.use_secrets(secrets=secrets)

KeyError: 'SCENARIOS_SECRETS'

We can now call `head` without passing in `secrets` and the engine will happily execute and show us a preview of the data.

In [6]:
dataflow.head(5)

MissingSecretsError: Required secrets are missing. Please call use_secrets to register the missing secrets.
Missing secrets:
https://dpreptestfiles.blob.core.windows.net/testfiles/read_csv_duplicate_headers.csv